In [10]:
import tensorflow as tf
import pandas as pd
import numpy as np

In [11]:
df = pd.read_pickle('2018_5000.pkl')
df = df[df['winner'] != '1/2']

In [12]:
layouts = df['layout'].to_list()
fix = np.dstack(layouts)
fix = np.rollaxis(fix,-1)
target = df['winner'].astype('int')

In [13]:
from sklearn.model_selection import train_test_split
x_train, x_test, y_train, y_test = train_test_split(fix,target, test_size=0.2)

In [14]:
x_train.shape

(241248, 8, 8)

In [ ]:
from tensorflow.keras.layers import Input, SimpleRNN, GRU, LSTM, Dense, Flatten, GlobalMaxPool1D, Dropout
from tensorflow.keras.models import Model
from tensorflow.keras.optimizers import SGD, Adam
i = Input(shape=x_train[0].shape)
x = LSTM(512, return_sequences=True)(i) #number of hidden units. This is one of the parameters that are guessed/experimented with
x = GlobalMaxPool1D()(x)
x = Dense(512, activation = 'relu')(x)
x = Dropout(0.2)(x)
x = Dense(512, activation = 'relu')(x)
x = Dropout(0.3)(x)
x = Dense(1, activation='sigmoid')(x)
model = Model(i, x)
model.summary()
model.compile(optimizer='adam',
              loss='binary_crossentropy',
              metrics=['accuracy'])
r = model.fit(x_train, y_train, validation_data=(x_test, y_test), epochs=20)

Model: "model_1"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
input_2 (InputLayer)         [(None, 8, 8)]            0         
_________________________________________________________________
lstm_1 (LSTM)                (None, 8, 512)            1067008   
_________________________________________________________________
global_max_pooling1d_1 (Glob (None, 512)               0         
_________________________________________________________________
dense_5 (Dense)              (None, 512)               262656    
_________________________________________________________________
dropout_4 (Dropout)          (None, 512)               0         
_________________________________________________________________
dense_6 (Dense)              (None, 512)               262656    
_________________________________________________________________
dropout_5 (Dropout)          (None, 512)               0   

In [ ]:
# Plot loss per iteration
import matplotlib.pyplot as plt
plt.plot(r.history['loss'], label='loss')
plt.plot(r.history['val_loss'], label='val_loss')
plt.legend()

In [ ]:
# Plot accuracy per iteration
plt.plot(r.history['accuracy'], label='acc')
plt.plot(r.history['val_accuracy'], label='val_acc')
plt.legend()

In [ ]:
model.save('chess_5000_2.h5')